In [226]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

def sigmoid (X):
    return 1 / (1 + np.exp(-X))

def d_sigmoid (X):
    return X * (1 - X)

def relu (X):
    return np.maximum(0, X)

def d_relu (X):
    return np.where(X > 0, 1, 0)

class MLP:
    def __init__ (self, size, learning_rate=10):
        self.size = size
        self.learning_rate = float(learning_rate)
        self.dep_num = len(size)
        self.activation_fun = ["sigmoid"] * self.dep_num
        self.h = [0] * self.dep_num
        self.w = []
        self.b = []
        for i in range(self.dep_num - 1):
            self.w.append(np.random.normal(scale=0.01, size=(self.size[i], self.size[i + 1])))
            self.b.append(np.zeros(self.size[i + 1]))
    
    def forward (self, X):
        self.h = [0] * self.dep_num
        self.h[0] = X
        for i in range(self.dep_num - 1):
            fun = sigmoid
            if (self.activation_fun[i] == "sigmoid"):
                fun = sigmoid
            if (self.activation_fun[i] == "relu"):
                fun = relu
            self.h[i + 1] = fun(np.dot(self.h[i], self.w[i]) + self.b[i])
        return self.h[self.dep_num - 1]
    
    def backward (self, y, y_pred):
        d_h = [0] * self.dep_num
        d_w = [0] * (self.dep_num - 1)
        d_b = [0] * (self.dep_num - 1)
        d_h[self.dep_num - 1] = -(y - y_pred)
        for i in range(self.dep_num - 2, -1, -1):
            fun = d_sigmoid
            if (self.activation_fun[i] == "sigmoid"):
                fun = d_sigmoid
            if (self.activation_fun[i] == "relu"):
                fun = d_relu
            d_w[i] = np.dot(self.h[i].T, d_h[i + 1] * fun(self.h[i + 1]))
            d_b[i] = np.sum(d_h[i + 1] * fun(self.h[i + 1]), axis=0)
            d_h[i] = np.dot(d_h[i + 1] * fun(self.h[i + 1]), self.w[i].T)
        for i in range(self.dep_num - 1):
            self.w[i] -= self.learning_rate * d_w[i]
            self.b[i] -= self.learning_rate * d_b[i]


def cross_entropy_loss (y_pred, y):
    return -np.mean(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))


In [227]:
train_data = pd.read_csv("./TrainData.csv")

X = train_data.drop(columns=["h1n1_vaccine", "seasonal_vaccine"])
y = train_data[["h1n1_vaccine", "seasonal_vaccine"]]
X, y = np.array(X, dtype=float), np.array(y, dtype=float)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
y1_train, y2_train = y_train[:, 0], y_train[:, 1]
y1_test, y2_test = y_test[:, 0], y_test[:, 1]

In [228]:
model1_relu = MLP(size=[64, 10, 2, 1], learning_rate=0.001)
model2_relu = MLP(size=[64, 50, 10, 1], learning_rate=0.001)
model1_relu.activation_fun = ["relu", "relu", "sigmoid"]
model2_relu.activation_fun = ["relu", "relu", "sigmoid"]

for i in range(1000):
    y1_pred = model1_relu.forward(X_train)
    loss1 = cross_entropy_loss(y1_pred, y1_train.reshape(-1, 1))
    model1_relu.backward(y1_train.reshape(-1, 1), y1_pred)
    y2_pred = model2_relu.forward(X_train)
    loss2 = cross_entropy_loss(y2_pred, y2_train.reshape(-1, 1))
    model2_relu.backward(y2_train.reshape(-1, 1), y2_pred)
    if (i % 100 == 0):
        print("Epoch", i)
        print("h1n1_vaccine loss:", loss1)
        print("seasonal_vaccine loss:", loss2)

Epoch 0
h1n1_vaccine loss: 0.6931520733646798
seasonal_vaccine loss: 0.6931462067705014
Epoch 100
h1n1_vaccine loss: 0.5167784048726672
seasonal_vaccine loss: 0.6979752854392599
Epoch 200
h1n1_vaccine loss: 0.5167784046597855
seasonal_vaccine loss: 0.6652514016404656
Epoch 300
h1n1_vaccine loss: 0.5167784045818881
seasonal_vaccine loss: 0.6907574872429145
Epoch 400
h1n1_vaccine loss: 0.5167784045277353
seasonal_vaccine loss: 0.690698880660715
Epoch 500
h1n1_vaccine loss: 0.5167784044490907
seasonal_vaccine loss: 0.681484096094743
Epoch 600
h1n1_vaccine loss: 0.5167784043463801
seasonal_vaccine loss: 0.7122364961702783
Epoch 700
h1n1_vaccine loss: 0.5167784036003973
seasonal_vaccine loss: 0.6180768134989043
Epoch 800
h1n1_vaccine loss: 0.5167783990703562
seasonal_vaccine loss: 0.6259047417939778
Epoch 900
h1n1_vaccine loss: 0.516778391605003
seasonal_vaccine loss: 0.5593838174298108


In [229]:
print("relu:")
print("h1n1_vaccine:", roc_auc_score(y1_test.reshape(-1, 1), model1_relu.forward(X_test)))
print("seasonal_vaccine:", roc_auc_score(y2_test.reshape(-1, 1), model2_relu.forward(X_test)))

relu:
h1n1_vaccine: 0.5171419681380683
seasonal_vaccine: 0.7777182090950866


In [230]:
model1_sigmoid = MLP(size=[64, 10, 1], learning_rate=0.001)
model2_sigmoid = MLP(size=[64, 10, 1], learning_rate=0.001)

for i in range(1000):
    y1_pred = model1_sigmoid.forward(X_train)
    loss1 = cross_entropy_loss(y1_pred, y1_train.reshape(-1, 1))
    model1_sigmoid.backward(y1_train.reshape(-1, 1), y1_pred)
    y2_pred = model2_sigmoid.forward(X_train)
    loss2 = cross_entropy_loss(y2_pred, y2_train.reshape(-1, 1))
    model2_sigmoid.backward(y2_train.reshape(-1, 1), y2_pred)
    if (i % 100 == 0):
        print("Epoch", i)
        print("h1n1_vaccine loss:", loss1)
        print("seasonal_vaccine loss:", loss2)

Epoch 0
h1n1_vaccine loss: 0.69284718437746
seasonal_vaccine loss: 0.6944521107743058
Epoch 100
h1n1_vaccine loss: 0.4228218186893261
seasonal_vaccine loss: 0.5899451449818709
Epoch 200
h1n1_vaccine loss: 0.4363561735817091
seasonal_vaccine loss: 0.6562026012874369
Epoch 300
h1n1_vaccine loss: 0.4190691780007851
seasonal_vaccine loss: 0.5743660190737676
Epoch 400
h1n1_vaccine loss: 0.42316599848995584
seasonal_vaccine loss: 0.5543263567920169
Epoch 500
h1n1_vaccine loss: 0.41554577802423653
seasonal_vaccine loss: 0.5417711164365895
Epoch 600
h1n1_vaccine loss: 0.40971324476602605
seasonal_vaccine loss: 0.5328943938317894
Epoch 700
h1n1_vaccine loss: 0.4052689339200382
seasonal_vaccine loss: 0.5250432751714676
Epoch 800
h1n1_vaccine loss: 0.40188944731069626
seasonal_vaccine loss: 0.519349339002872
Epoch 900
h1n1_vaccine loss: 0.3991974336914678
seasonal_vaccine loss: 0.5153369561404186


In [231]:
print("sigmoid:")
print("h1n1_vaccine:", roc_auc_score(y1_test.reshape(-1, 1), model1_sigmoid.forward(X_test)))
print("seasonal_vaccine:", roc_auc_score(y2_test.reshape(-1, 1), model2_sigmoid.forward(X_test)))

sigmoid:
h1n1_vaccine: 0.8120809824095586
seasonal_vaccine: 0.8252311164772984


In [ ]:
# 经过多次尝试发现，在使用 relu 激活函数的模型不如使用 sigmoid 激活函数的模型，大部分时候使用 relu 激活函数的模型甚至没有任何效果

In [237]:
test_features = pd.read_csv("./TestFeatures.csv")
id = np.array(test_features["respondent_id"])
X_features = np.array(test_features.drop(columns=["respondent_id"]), dtype=float)

y1_label = model1_sigmoid.forward(X_features).reshape(1, -1)[0]
y2_label = model2_sigmoid.forward(X_features).reshape(1, -1)[0]

output = pd.DataFrame({"respondent_id": id, "h1n1_vaccine": y1_label, "seasonal_vaccine": y2_label})
output.to_csv("./submission.csv", index=False)